# Networking (Polyglot)

In [ ]:
#!import ../../lib/fsharp/Notebooks.dib
#!import ../../lib/fsharp/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/argu/6.2.2/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"

In [ ]:
#!import ../../lib/spiral/common.fsx
#!import ../../lib/spiral/sm.fsx
#!import ../../lib/spiral/date_time.fsx
#!import ../../lib/spiral/file_system.fsx
#!import ../../lib/spiral/trace.fsx
#!import ../../lib/spiral/lib.fsx
#!import ../../lib/fsharp/Common.fs
#!import ../../lib/fsharp/CommonFSharp.fs
#!import ../../lib/fsharp/Async.fs
#!import ../../lib/fsharp/Runtime.fs

In [ ]:
#if !INTERACTIVE
open Lib
#endif

In [ ]:
open Common

## testPortOpen

In [ ]:
let inline testPortOpen port = async {
    let! ct = Async.CancellationToken
    use client = new System.Net.Sockets.TcpClient ()
    try
        do! client.ConnectAsync ("127.0.0.1", port, ct) |> Async.awaitValueTaskUnit
        return true
    with ex ->
        trace Verbose (fun () -> $"testPortOpen / ex: {ex |> SpiralSm.format_exception}") getLocals
        return false
}

In [ ]:
//// test

testPortOpen 65536
|> Async.runWithTimeout 120
|> _assertEqual (Some false)

00:00:00 #1 [verbose] testPortOpen / ex: System.ArgumentOutOfRangeException: Specified argument was out of the range of valid values. (Parameter 'port')
FSharpOption<Boolean>
      Value: False


In [ ]:
let inline testPortOpenTimeout timeout port = async {
    let! result =
        testPortOpen port
        |> Async.runWithTimeoutAsync timeout
    return
        match result with
        | None -> false
        | Some result -> result
}

In [ ]:
//// test

testPortOpenTimeout 120 65535
|> Async.RunSynchronously
|> _assertEqual false

00:00:00 #2 [debug] runWithTimeoutChildAsync / timeout: 120
False


## waitForPortAccess

In [ ]:
let inline waitForPortAccess timeout status port =
    let rec loop retry = async {
        let! isPortOpen =
            match timeout with
            | None -> testPortOpen port
            | Some timeout -> testPortOpenTimeout timeout port
        if isPortOpen = status
        then return retry
        else
            if retry % 100 = 0 then
                let getLocals () = $"port: {port} / retry: {retry} / {getLocals ()}"
                trace Verbose (fun () -> "waitForPortAccess") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let port = 5555

let inline lockPort () = async {
    trace Debug (fun () -> "_1") getLocals
    do! Async.Sleep 5000
    use listener = new System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    trace Debug (fun () -> "_2") getLocals
    listener.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_4") getLocals
    listener.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPort () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! retries1 = waitForPortAccess None true port
    trace Debug (fun () -> "4") getLocals
    let! retries2 = waitForPortAccess None false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries1, retries2
}
|> Async.runWithTimeout 20000
|> function
    | Some (retries1, retries2) ->
        retries1
        |> _isBetween
            (if Runtime.isWindows () then 2 else 2)
            (if Runtime.isWindows () then 5 else 1500)

        retries2
        |> _isBetween
            (if Runtime.isWindows () then 80 else 80)
            (if Runtime.isWindows () then 150 else 600)

        true
    | _ -> false
|> _assertEqual true

00:00:01 #3 [debug] 1
00:00:01 #4 [debug] _1
00:00:01 #5 [debug] 2
00:00:01 #6 [debug] 3
00:00:03 #7 [verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
00:00:03 #8 [verbose] waitForPortAccess / port: 5555 / retry: 0
00:00:05 #9 [verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
00:00:06 #10 [debug] _2
00:00:06 #11 [debug] _3
00:00:06 #12 [debug] 4
00:00:06 #13 [verbose] waitForPortAccess / port: 5555 / retry: 0
00:00:07 #14 [verbose] waitForPortAccess / port: 5555 / retry: 100
00:00:08 #15 [debug] _4
00:00:08 #16 [debug] _5
00:00:10 #17 [verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
00:00:10 #18 [debug] 5
00:00:10 #19 [debug] 6
2
2
2
118
118
118
True


In [ ]:
//// test

let port = 5555

let inline lockPort () = async {
    trace Debug (fun () -> "_1") getLocals
    do! Async.Sleep 500
    use listener = new System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    trace Debug (fun () -> "_2") getLocals
    listener.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 200
    trace Debug (fun () -> "_4") getLocals
    listener.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPort () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! retries1 = waitForPortAccess (Some 60) true port
    trace Debug (fun () -> "4") getLocals
    let! retries2 = waitForPortAccess (Some 60) false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries1, retries2
}
|> Async.runWithTimeout 2000
|> function
    | Some (retries1, retries2) ->
        retries1
        |> _isBetween
            (if Runtime.isWindows () then 4 else 2)
            (if Runtime.isWindows () then 15 else 150)

        retries2
        |> _isBetween
            (if Runtime.isWindows () then 5 else 0)
            (if Runtime.isWindows () then 20 else 60)

        true
    | _ -> false
|> _assertEqual true

00:00:10 #20 [debug] 1
00:00:10 #21 [debug] 2
00:00:10 #22 [debug] _1
00:00:10 #23 [debug] 3
00:00:10 #24 [debug] runWithTimeoutChildAsync / timeout: 60
00:00:10 #25 [verbose] waitForPortAccess / port: 5555 / retry: 0
00:00:10 #26 [debug] runWithTimeoutChildAsync / timeout: 60
00:00:11 #27 [debug] runWithTimeoutChildAsync / timeout: 60
00:00:11 #28 [debug] runWithTimeoutChildAsync / timeout: 60
00:00:11 #29 [debug] runWithTimeoutChildAsync / timeout: 60
00:00:11 #30 [debug] runWithTimeoutChildAsync / timeout: 60
00:00:11 #31 [debug] _2
00:00:11 #32 [debug] _3
00:00:11 #33 [debug] runWithTimeoutChildAsync / timeout: 60
00:00:11 #34 [debug] 4
00:00:11 #35 [verbose] waitForPortAccess / port: 5555 / retry: 0
00:00:11 #36 [debug] _4
00:00:11 #37 [debug] _5
00:00:11 #38 [debug] runWithTimeoutChildAsync / timeout: 60
00:00:11 #39 [debug] 5
00:00:11 #40 [debug] 6
7
7
7
8
8
8
True


## getAvailablePort

In [ ]:
let inline getAvailablePort timeout initialPort =
    let rec loop port = async {
        let! isPortOpen =
            match timeout with
            | None -> testPortOpen port
            | Some timeout -> testPortOpenTimeout timeout port
        if not isPortOpen
        then return port
        else return! loop (port + 1)
    }
    loop initialPort

In [ ]:
//// test

let port = 5555

let inline lockPorts () = async {
    trace Debug (fun () -> "_1") getLocals
    use listener1 = new System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    use listener2 = new System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port + 1)
    trace Debug (fun () -> "_2") getLocals
    listener1.Start ()
    listener2.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 4000
    trace Debug (fun () -> "_4") getLocals
    listener1.Stop ()
    listener2.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPorts () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 240
    trace Debug (fun () -> "3") getLocals
    let! availablePort = getAvailablePort None port
    trace Debug (fun () -> "4") getLocals
    let! retries = waitForPortAccess None false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return availablePort, retries
}
|> Async.runWithTimeout 15000
|> function
    | Some (availablePort, retries) ->
        availablePort |> _assertEqual (port + 2)

        retries
        |> _isBetween
            (if Runtime.isWindows () then 100 else 100)
            (if Runtime.isWindows () then 150 else 1200)

        true
    | _ -> false
|> _assertEqual true

00:00:12 #41 [debug] 1
00:00:12 #42 [debug] 2
00:00:12 #43 [debug] _1
00:00:12 #44 [debug] _2
00:00:12 #45 [debug] _3
00:00:12 #46 [debug] 3
00:00:14 #47 [verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
00:00:14 #48 [debug] 4
00:00:14 #49 [verbose] waitForPortAccess / port: 5555 / retry: 0
00:00:16 #50 [verbose] waitForPortAccess / port: 5555 / retry: 100
00:00:16 #51 [debug] _4
00:00:16 #52 [debug] _5
00:00:18 #53 [verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
00:00:18 #54 [debug] 5
00:00:18 #55 [debug] 6
5557
109
109
109
True


In [ ]:
//// test

let port = 5555

let inline lockPorts () = async {
    trace Debug (fun () -> "_1") getLocals
    use listener1 = new System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    use listener2 = new System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port + 1)
    trace Debug (fun () -> "_2") getLocals
    listener1.Start ()
    listener2.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 400
    trace Debug (fun () -> "_4") getLocals
    listener1.Stop ()
    listener2.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPorts () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 240
    trace Debug (fun () -> "3") getLocals
    let! availablePort = getAvailablePort (Some 60) port
    trace Debug (fun () -> "4") getLocals
    let! retries = waitForPortAccess (Some 60) false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return availablePort, retries
}
|> Async.runWithTimeout 1500
|> function
    | Some (availablePort, retries) ->
        availablePort |> _assertEqual (port + 2)

        retries
        |> _isBetween
            (if Runtime.isWindows () then 2 else 1)
            (if Runtime.isWindows () then 10 else 120)

        true
    | _ -> false
|> _assertEqual true

00:00:18 #56 [debug] 1
00:00:18 #57 [debug] 2
00:00:18 #58 [debug] _1
00:00:18 #59 [debug] _2
00:00:18 #60 [debug] _3
00:00:18 #61 [debug] 3
00:00:19 #62 [debug] runWithTimeoutChildAsync / timeout: 60
00:00:19 #63 [debug] 4
00:00:19 #64 [verbose] waitForPortAccess / port: 5555 / retry: 0
00:00:19 #65 [debug] _4
00:00:19 #66 [debug] _5
00:00:19 #67 [debug] runWithTimeoutChildAsync / timeout: 60
00:00:19 #68 [debug] 5
00:00:19 #69 [debug] 6
5557
5
5
5
True
